# Tema 5 - Modele de regresie

In [1]:
import numpy as np
import pandas as pd
import sklearn

print('At the moment the program was made:')
print('\tNumPy Version = 1.20.2')
print('\tScykit_learn Version = 0.24.1')
print('\tPandas Version = 1.2.3\n')

print(f'Current NumPy Version = {np.__version__}')
print(f'Current Sklearn Version = {sklearn.__version__}')
print(f'Current Pandas Version = {pd.__version__}')

At the moment the program was made:
	NumPy Version = 1.20.2
	Scykit_learn Version = 0.24.1
	Pandas Version = 1.2.3

Current NumPy Version = 1.20.2
Current Sklearn Version = 0.24.1
Current Pandas Version = 1.2.4


In [2]:
zeros = np.zeros(10)
CPU_model_df = pd.DataFrame({
    'Model':list(zeros), 
    'Search_strategy': list(zeros), 
    'test_neg_mean_absolute_error': list(zeros), 
    'test_neg_mean_squared_error': list(zeros), 
    'test_neg_median_absolute_error': list(zeros),
    'train_neg_mean_absolute_error': list(zeros),
    'train_neg_mean_squared_error': list(zeros),
    'train_neg_median_absolute_error': list(zeros),
    'fit_time': list(zeros),
    'score_time': list(zeros)
})

## CPU Computer Hardware

* CPU Computer Hardware; excludeti din dataset coloanele: vendor name, model name, estimated relative performance; se va estima coloana "published relative performance".

In [44]:
CPU_data = pd.read_csv("data/machine.data", delimiter=",", header=None)
CPU_data = CPU_data.drop([0, 1, 9], axis = 1)
#CPU_data = CPU_data.drop(1, axis = 1)
#CPU_data = CPU_data.drop(9, axis = 1)
CPU_data.head()

,2,3,4,5,6,7,8
0,125,256,6000,256,16,128,198
1,29,8000,32000,32,8,32,269
2,29,8000,32000,32,8,32,220
3,29,8000,32000,32,8,32,172
4,29,8000,16000,32,8,16,132


In [45]:
CPU_X = CPU_data.iloc[:, :-1].values
CPU_y = CPU_data.iloc[:, -1].values

In [46]:
def populate_dataFrame(dataFrame, scores, regression_model, grid_search, index):
    dataFrame.iloc[index, 0] = regression_model
    dataFrame.iloc[index, 1] = grid_search
    dataFrame.iloc[index, 2] = scores['test_neg_mean_absolute_error'].mean()
    dataFrame.iloc[index, 3] = scores['test_neg_mean_squared_error'].mean()
    dataFrame.iloc[index, 4] = scores['test_neg_median_absolute_error'].mean()
    dataFrame.iloc[index, 5] = scores['train_neg_mean_absolute_error'].mean()
    dataFrame.iloc[index, 6] = scores['train_neg_mean_squared_error'].mean()
    dataFrame.iloc[index, 7] = scores['train_neg_median_absolute_error'].mean()
    dataFrame.iloc[index, 8] = scores['fit_time'].mean()
    dataFrame.iloc[index, 9] = scores['score_time'].mean()


### LinearRegression

In [47]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn import metrics

In [48]:
regression = LinearRegression()
parameter_grid = {"fit_intercept":[True, False], "normalize":[True, False], "n_jobs":[None, 2, 4, 6]}

In [49]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(CPU_X, CPU_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'fit_intercept': True, 'n_jobs': None, 'normalize': False}


In [50]:
tunned_regression = LinearRegression(**best_parameters)
grid_scores = cross_validate(tunned_regression, CPU_X, CPU_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(CPU_model_df, grid_scores, 'LinearRegression', 'GridSearchCV', 0)

In [51]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(CPU_X, CPU_y)
best_parameters = random_search.best_params_
print(best_parameters)

{'normalize': False, 'n_jobs': 6, 'fit_intercept': True}


In [52]:
tunned_regression = LinearRegression(**best_parameters)
random_scores = cross_validate(tunned_regression, CPU_X, CPU_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(CPU_model_df, random_scores, 'LinearRegression', 'RandomizedSearchCV', 1)
CPU_model_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,-43.378262,-6383.656697,-27.053792,-36.695674,-3243.698611,-25.581576,0.001137,0.001656
1,LinearRegression,RandomizedSearchCV,-43.378262,-6383.656697,-27.053792,-36.695674,-3243.698611,-25.581576,0.001380,0.001835
2,DecisionTreeRegressor,GridSearchCV,-57.389000,-14056.576438,-24.004719,-48.668666,-11086.309265,-19.687989,0.000726,0.001235
3,DecisionTreeRegressor,RandomizedSearchCV,-63.579477,-15322.530667,-29.370136,-49.608063,-11677.636746,-19.533689,0.000515,0.001788
4,KernelRidge,GridSearchCV,-47.222355,-8067.967074,-30.918495,-39.472751,-3987.727118,-24.955848,0.002241,0.001880
5,KernelRidge,RandomizedSearchCV,-47.222355,-8067.967074,-30.918495,-39.472751,-3987.727118,-24.955848,0.001698,0.002739
6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### DecisionTreeRegressor

In [53]:
from sklearn import tree

In [54]:
regression = tree.DecisionTreeRegressor()
parameter_grid = {'splitter':['best', 'random'], 'max_depth': [1, 3, 5, 7], 'min_samples_leaf':[1, 2, 3, 4, 5],
                 'min_weight_fraction_leaf':[0.1, 0.2, 0.3, 0.4, 0.5], 'max_features':['auto', 'log2', 'sqrt', None],
                 'max_leaf_nodes':[None, 10, 20, 30, 40, 50]}

In [55]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(CPU_X, CPU_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'max_depth': 3, 'max_features': 'sqrt', 'max_leaf_nodes': 50, 'min_samples_leaf': 2, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}


In [56]:
tuned_regression = tree.DecisionTreeRegressor(**best_parameters)
grid_scores = cross_validate(tuned_regression, CPU_X, CPU_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(CPU_model_df, grid_scores, 'DecisionTreeRegressor', 'GridSearchCV', 2)

In [58]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(CPU_X, CPU_y)
best_parameters = random_search.best_params_
print(best_parameters)

{'splitter': 'best', 'min_weight_fraction_leaf': 0.1, 'min_samples_leaf': 5, 'max_leaf_nodes': None, 'max_features': 'auto', 'max_depth': 7}


In [59]:
tuned_regression = tree.DecisionTreeRegressor(**best_parameters)
random_scores = cross_validate(tuned_regression, CPU_X, CPU_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(CPU_model_df, random_scores, 'DecisionTreeRegressor', 'RandomizedSearchCV', 3)

CPU_model_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,-43.378262,-6383.656697,-27.053792,-36.695674,-3243.698611,-25.581576,0.001137,0.001656
1,LinearRegression,RandomizedSearchCV,-43.378262,-6383.656697,-27.053792,-36.695674,-3243.698611,-25.581576,0.001380,0.001835
2,DecisionTreeRegressor,GridSearchCV,-54.212925,-13093.943610,-24.963985,-48.268988,-11082.913760,-18.630747,0.002394,0.002762
3,DecisionTreeRegressor,RandomizedSearchCV,-52.556074,-12328.675240,-22.768793,-43.594256,-10461.485505,-13.221429,0.001063,0.001462
4,KernelRidge,GridSearchCV,-47.222355,-8067.967074,-30.918495,-39.472751,-3987.727118,-24.955848,0.002241,0.001880
5,KernelRidge,RandomizedSearchCV,-47.222355,-8067.967074,-30.918495,-39.472751,-3987.727118,-24.955848,0.001698,0.002739
6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### KernelRidge

In [60]:
from sklearn.kernel_ridge import KernelRidge

In [61]:
regression = KernelRidge()
parameter_grid = {'kernel':['linear', 'rbf'], 'alpha':[1, 2, 3, 5], 'gamma':[None, 0.1, 0.001]}

In [62]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(CPU_X, CPU_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'alpha': 5, 'gamma': None, 'kernel': 'linear'}


In [63]:
tuned_regression = KernelRidge(**best_parameters)
grid_scores = cross_validate(tuned_regression, CPU_X, CPU_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(CPU_model_df, grid_scores, 'KernelRidge', 'GridSearchCV', 4)

In [64]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(CPU_X, CPU_y)
best_parameters = random_search.best_params_
print(best_parameters)

{'kernel': 'linear', 'gamma': 0.1, 'alpha': 5}


In [65]:
tuned_regression = KernelRidge(**best_parameters)
random_scores = cross_validate(tuned_regression, CPU_X, CPU_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(CPU_model_df, random_scores, 'KernelRidge', 'RandomizedSearchCV', 5)

CPU_model_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,-43.378262,-6383.656697,-27.053792,-36.695674,-3243.698611,-25.581576,0.001137,0.001656
1,LinearRegression,RandomizedSearchCV,-43.378262,-6383.656697,-27.053792,-36.695674,-3243.698611,-25.581576,0.001380,0.001835
2,DecisionTreeRegressor,GridSearchCV,-54.212925,-13093.943610,-24.963985,-48.268988,-11082.913760,-18.630747,0.002394,0.002762
3,DecisionTreeRegressor,RandomizedSearchCV,-52.556074,-12328.675240,-22.768793,-43.594256,-10461.485505,-13.221429,0.001063,0.001462
4,KernelRidge,GridSearchCV,-47.222355,-8067.967074,-30.918495,-39.472751,-3987.727118,-24.955848,0.002908,0.002034
5,KernelRidge,RandomizedSearchCV,-47.222355,-8067.967074,-30.918495,-39.472751,-3987.727118,-24.955848,0.002340,0.001950
6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### GaussianProcessRegressor

In [76]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, DotProduct

In [77]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
CPU_X_scaled = scaler.fit_transform(CPU_X)
print(CPU_X_scaled)

[[0.07282535 0.00601202 0.09284284 1.         0.30769231 0.72727273]
 [0.00809171 0.24849699 0.4994995  0.125      0.15384615 0.18181818]
 [0.00809171 0.24849699 0.4994995  0.125      0.15384615 0.18181818]
 ...
 [0.07282535 0.06062124 0.12412412 0.         0.03846154 0.07954545]
 [0.31220499 0.01402806 0.12412412 0.125      0.         0.        ]
 [0.31220499 0.02930862 0.06156156 0.         0.         0.        ]]


In [78]:
regression = GaussianProcessRegressor()
parameter_grid = [{"alpha":[1e-2, 1e-3], "kernel":[RBF(l) for l in np.logspace(-1, 1, 2)]},
                 {"alpha":[1e-2, 1e-3], "kernel": [DotProduct(sigma_0) for sigma_0 in np.logspace(-1, 1, 2)]}]

In [79]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(CPU_X_scaled, CPU_y)
best_parameters = grid_search.best_params_
print(best_parameters)

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\gaussian_process\kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\gaussian_process\kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\gaussian_process\kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit a

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\gaussian_process\_gpr.py:509: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\gaussian_process\_gpr.py:509: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the dat

{'alpha': 0.01, 'kernel': DotProduct(sigma_0=0.1)}


C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "


In [80]:
tuned_regression = GaussianProcessRegressor(**best_parameters)
grid_scores = cross_validate(tuned_regression, CPU_X_scaled, CPU_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(CPU_model_df, grid_scores, 'GaussianProcessRegressor', 'GridSearchCV', 6)

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\gaussian_process\_gpr.py:509: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\gaussian_process\_gpr.py:509: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the dat

In [81]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(CPU_X_scaled, CPU_y)
best_parameters = random_search.best_params_
print(best_parameters)

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\gaussian_process\kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\gaussian_process\kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
C:\Users\Laur\an

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\gaussian_process\_gpr.py:509: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\gaussian_process\_gpr.py:509: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the dat

{'kernel': DotProduct(sigma_0=0.1), 'alpha': 0.01}


C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\gaussian_process\_gpr.py:509: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "


In [82]:
tuned_regression = GaussianProcessRegressor(**best_parameters)
random_scores = cross_validate(tuned_regression, CPU_X_scaled, CPU_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(CPU_model_df, random_scores, 'GaussianProcessRegressor', 'RandomizedSearchCV', 7)

CPU_model_df.head(n=10)

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\gaussian_process\_gpr.py:509: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\gaussian_process\_gpr.py:509: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the dat

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,-43.378262,-6383.656697,-27.053792,-36.695674,-3243.698611,-25.581576,0.001137,0.001656
1,LinearRegression,RandomizedSearchCV,-43.378262,-6383.656697,-27.053792,-36.695674,-3243.698611,-25.581576,0.001380,0.001835
2,DecisionTreeRegressor,GridSearchCV,-54.212925,-13093.943610,-24.963985,-48.268988,-11082.913760,-18.630747,0.002394,0.002762
3,DecisionTreeRegressor,RandomizedSearchCV,-52.556074,-12328.675240,-22.768793,-43.594256,-10461.485505,-13.221429,0.001063,0.001462
4,KernelRidge,GridSearchCV,-47.222355,-8067.967074,-30.918495,-39.472751,-3987.727118,-24.955848,0.002908,0.002034
5,KernelRidge,RandomizedSearchCV,-47.222355,-8067.967074,-30.918495,-39.472751,-3987.727118,-24.955848,0.002340,0.001950
6,GaussianProcessRegressor,GridSearchCV,-43.327007,-6367.059231,-27.077930,-36.664309,-3243.095266,-25.468359,0.134010,0.002576
7,GaussianProcessRegressor,RandomizedSearchCV,-43.327007,-6367.059231,-27.077930,-36.664309,-3243.095266,-25.468359,0.095678,0.001571
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### LogisticRegression

In [90]:
from sklearn.linear_model import LogisticRegression

In [91]:
regression = LogisticRegression()
parameter_grid = {'penalty': ['l1', 'l2'], 'C': np.logspace(-4, 4, 20), 'solver':['liblinear'], 'max_iter':[1e4]}

In [92]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(CPU_X_scaled, CPU_y)
best_parameters = grid_search.best_params_
print(best_parameters)

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


{'C': 1438.44988828766, 'max_iter': 10000.0, 'penalty': 'l2', 'solver': 'liblinear'}


In [93]:
tuned_regression = LogisticRegression(**best_parameters)
grid_scores = cross_validate(tuned_regression, CPU_X_scaled, CPU_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(CPU_model_df, grid_scores, 'LogisticRegression', 'GridSearchCV', 8)

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [94]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(CPU_X_scaled, CPU_y)
best_parameters = random_search.best_params_
print(best_parameters)

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


{'solver': 'liblinear', 'penalty': 'l2', 'max_iter': 10000.0, 'C': 78.47599703514607}


In [95]:
tuned_regression = LogisticRegression(**best_parameters)
random_scores = cross_validate(tuned_regression, CPU_X_scaled, CPU_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(CPU_model_df, random_scores, 'LogisticRegression', 'RandomizedSearchCV', 9)

CPU_model_df.head(n=10)

C:\Users\Laur\anaconda3\envs\IntroducereInDataScience\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,-43.378262,-6383.656697,-27.053792,-36.695674,-3243.698611,-25.581576,0.001137,0.001656
1,LinearRegression,RandomizedSearchCV,-43.378262,-6383.656697,-27.053792,-36.695674,-3243.698611,-25.581576,0.001380,0.001835
2,DecisionTreeRegressor,GridSearchCV,-54.212925,-13093.943610,-24.963985,-48.268988,-11082.913760,-18.630747,0.002394,0.002762
3,DecisionTreeRegressor,RandomizedSearchCV,-52.556074,-12328.675240,-22.768793,-43.594256,-10461.485505,-13.221429,0.001063,0.001462
4,KernelRidge,GridSearchCV,-47.222355,-8067.967074,-30.918495,-39.472751,-3987.727118,-24.955848,0.002908,0.002034
5,KernelRidge,RandomizedSearchCV,-47.222355,-8067.967074,-30.918495,-39.472751,-3987.727118,-24.955848,0.002340,0.001950
6,GaussianProcessRegressor,GridSearchCV,-43.327007,-6367.059231,-27.077930,-36.664309,-3243.095266,-25.468359,0.134010,0.002576
7,GaussianProcessRegressor,RandomizedSearchCV,-43.327007,-6367.059231,-27.077930,-36.664309,-3243.095266,-25.468359,0.095678,0.001571
8,LogisticRegression,GridSearchCV,-47.049245,-8983.801394,-15.800000,-12.373460,-659.393449,-3.600000,0.110144,0.002071
9,LogisticRegression,RandomizedSearchCV,-47.498722,-9666.730314,-16.600000,-23.533305,-2255.969119,-10.000000,0.033905,0.000880


In [102]:
CPU_model_df.iloc[:, 2:] = np.abs(CPU_model_df.iloc[:, 2:])
CPU_model_df = CPU_model_df.rename(columns={'test_neg_mean_absolute_error': 'test_mean_absolute_error', 'test_neg_mean_squared_error': 'test_mean_squared_error', 
                                            'test_neg_median_absolute_error': 'test_median_absolute_error', 'train_neg_mean_absolute_error': 'train_mean_absolute_error', 
                                            'train_neg_mean_squared_error': 'train_mean_squared_error', 'train_neg_median_absolute_error': 'train_median_absolute_error'})
CPU_model_df.head(n=10)

,Model,Search_strategy,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,43.378262,6383.656697,27.053792,36.695674,3243.698611,25.581576,0.001137,0.001656
1,LinearRegression,RandomizedSearchCV,43.378262,6383.656697,27.053792,36.695674,3243.698611,25.581576,0.001380,0.001835
2,DecisionTreeRegressor,GridSearchCV,54.212925,13093.943610,24.963985,48.268988,11082.913760,18.630747,0.002394,0.002762
3,DecisionTreeRegressor,RandomizedSearchCV,52.556074,12328.675240,22.768793,43.594256,10461.485505,13.221429,0.001063,0.001462
4,KernelRidge,GridSearchCV,47.222355,8067.967074,30.918495,39.472751,3987.727118,24.955848,0.002908,0.002034
5,KernelRidge,RandomizedSearchCV,47.222355,8067.967074,30.918495,39.472751,3987.727118,24.955848,0.002340,0.001950
6,GaussianProcessRegressor,GridSearchCV,43.327007,6367.059231,27.077930,36.664309,3243.095266,25.468359,0.134010,0.002576
7,GaussianProcessRegressor,RandomizedSearchCV,43.327007,6367.059231,27.077930,36.664309,3243.095266,25.468359,0.095678,0.001571
8,LogisticRegression,GridSearchCV,47.049245,8983.801394,15.800000,12.373460,659.393449,3.600000,0.110144,0.002071
9,LogisticRegression,RandomizedSearchCV,47.498722,9666.730314,16.600000,23.533305,2255.969119,10.000000,0.033905,0.000880


In [115]:
# def highlight_max(data, color='red'):
#     attr = 'background-color: {}'.format(color)
#     if data.ndim == 1:
#         is_max = data == data.max()
#         return [attr if v else '' for v in is_max]
#     else:
#         is_max = data == data.max().max()
#         return pd.DataFrame(np.where(is_max, attr, ''), index = data.index, columns = data.columns)

def color_max(val, maxim):
    color = 'red' if val == maxim else 'white'
    return 'color: %s' % color

maxim = CPU_model_df['test_mean_absolute_error'].max()
for x in CPU_model_df['test_mean_absolute_error']:
    color_max(x, maxim)

CPU_model_df.head(n=10)
# column = CPU_model_df['test_mean_absolute_error']
# max_index = column.idxmax()

,Model,Search_strategy,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,43.378262,6383.656697,27.053792,36.695674,3243.698611,25.581576,0.001137,0.001656
1,LinearRegression,RandomizedSearchCV,43.378262,6383.656697,27.053792,36.695674,3243.698611,25.581576,0.001380,0.001835
2,DecisionTreeRegressor,GridSearchCV,54.212925,13093.943610,24.963985,48.268988,11082.913760,18.630747,0.002394,0.002762
3,DecisionTreeRegressor,RandomizedSearchCV,52.556074,12328.675240,22.768793,43.594256,10461.485505,13.221429,0.001063,0.001462
4,KernelRidge,GridSearchCV,47.222355,8067.967074,30.918495,39.472751,3987.727118,24.955848,0.002908,0.002034
5,KernelRidge,RandomizedSearchCV,47.222355,8067.967074,30.918495,39.472751,3987.727118,24.955848,0.002340,0.001950
6,GaussianProcessRegressor,GridSearchCV,43.327007,6367.059231,27.077930,36.664309,3243.095266,25.468359,0.134010,0.002576
7,GaussianProcessRegressor,RandomizedSearchCV,43.327007,6367.059231,27.077930,36.664309,3243.095266,25.468359,0.095678,0.001571
8,LogisticRegression,GridSearchCV,47.049245,8983.801394,15.800000,12.373460,659.393449,3.600000,0.110144,0.002071
9,LogisticRegression,RandomizedSearchCV,47.498722,9666.730314,16.600000,23.533305,2255.969119,10.000000,0.033905,0.000880


## Boston Housing

In [3]:
BH_data = pd.read_csv("data/housing.data", delimiter = "\t", header = None)
BH_data.head()

,0
0,0.00632 18.00 2.310 0 0.5380 6.5750 65...
1,0.02731 0.00 7.070 0 0.4690 6.4210 78...
2,0.02729 0.00 7.070 0 0.4690 7.1850 61...
3,0.03237 0.00 2.180 0 0.4580 6.9980 45...
4,0.06905 0.00 2.180 0 0.4580 7.1470 54...


## Wisconsin Breast Cancer

In [4]:
WBC_data = pd.read_csv("data/r_wpbc.data", delimiter = ",", header = None)
WBC_data.head()
assert not np.isnan(WBC_data.values.sum())
WBC_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 33 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       194 non-null    int64  
 1   1       194 non-null    float64
 2   2       194 non-null    float64
 3   3       194 non-null    float64
 4   4       194 non-null    float64
 5   5       194 non-null    float64
 6   6       194 non-null    float64
 7   7       194 non-null    float64
 8   8       194 non-null    float64
 9   9       194 non-null    float64
 10  10      194 non-null    float64
 11  11      194 non-null    float64
 12  12      194 non-null    float64
 13  13      194 non-null    float64
 14  14      194 non-null    float64
 15  15      194 non-null    float64
 16  16      194 non-null    float64
 17  17      194 non-null    float64
 18  18      194 non-null    float64
 19  19      194 non-null    float64
 20  20      194 non-null    float64
 21  21      194 non-null    float64
 22  22

## Comunities and Crime

In [5]:
com_crime_data = pd.read_csv("data/communities.data", delimiter = ",", header = None)
com_crime_data = com_crime_data.drop(range(5), axis = 1)
com_crime_data = com_crime_data.replace("?", np.nan)
com_crime_data = com_crime_data.dropna()
com_crime_data.head()

,5,6,7,8,9,10,11,12,13,14,...,118,119,120,121,122,123,124,125,126,127
0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,...,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
16,0.15,0.31,0.40,0.63,0.14,0.06,0.58,0.72,0.65,0.47,...,0.06,0.39,0.84,0.06,0.06,0.91,0.5,0.88,0.26,0.49
20,0.25,0.54,0.05,0.71,0.48,0.30,0.42,0.48,0.28,0.32,...,0.09,0.46,0.05,0.09,0.05,0.88,0.5,0.76,0.13,0.34
21,1.00,0.42,0.47,0.59,0.12,0.05,0.41,0.53,0.34,0.33,...,1.00,0.07,0.15,1,0.35,0.73,0,0.31,0.21,0.69
23,0.11,0.43,0.04,0.89,0.09,0.06,0.45,0.48,0.31,0.46,...,0.16,0.12,0.07,0.04,0.01,0.81,1,0.56,0.09,0.63


In [6]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression